<a href="https://colab.research.google.com/github/lIlIlIIIlIIl/DeepLearning_Basic/blob/main/2_005_XAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import CIFAR10, ImageNet, STL10
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary
import skimage.transform
import argparse

In [8]:
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()

args.batch_size = 64
args.epochs = 15

# Dataset

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = CIFAR10(root='../data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='../data', train=False, download=True, transform=transform)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [40000, 10000])


train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

print(f"Dataset size: {len(train_loader)}")
image, label = next(iter(train_loader))
print(f"Image shape: {image.shape}")

Files already downloaded and verified
Files already downloaded and verified
Dataset size: 625
Image shape: torch.Size([64, 3, 224, 224])


# Model

In [10]:
base_model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
base_model.classifier = nn.Identity()

class VGG16_with_GAP(nn.Module):
    def __init__(self):
        super(VGG16_with_GAP, self).__init__()
        self.base = base_model.features
        self.gap = nn.AvgPool2d(7)
        self.fc = nn.Linear(512, 10)
        self.softmax = nn.Softmax(dim=1)

        for param in list(self.base.parameters())[:-4]:
            param.requires_grad = False

    def forward(self, x):
        x = self.base(x)
        features = x.clone()
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.softmax(x)
        return x, features

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 75.9MB/s]


In [13]:
model = VGG16_with_GAP()

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters(),lr=0.00001)

VGG16_with_GAP(
  (base): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dil

# Train

In [ ]:
model.cuda()
for epoch in range(args.epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()

        outputs, _ = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset) * 100

    correct = 0
    total = 0
    examples = []


    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.cuda()
            labels = labels.cuda()
            outputs, _ = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels.data).sum().item()

    print(f'Epoch {str(epoch+1).zfill(2)}/{args.epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f} %, Validation Accuracy: {100 * correct / total}%')

In [ ]:
model_path = os.path.join('./', 'model.pth')
torch.save(model, model_path)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))